In [ ]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'

input_folder = os.path.join(BASE_DIR, '04_base_maestra')
# Crearemos una nueva carpeta para este producto intermedio de análisis
output_folder = os.path.join(BASE_DIR, '05_analisis_agregado') 
input_file = 'elecciones_manzanillo_master_2012_2024.csv'
output_file = 'analisis_agregado_por_seccion.csv'

# --- 2. CARGA DE LA BASE DE DATOS MAESTRA ---
file_path = os.path.join(input_folder, input_file)
try:
    master_df = pd.read_csv(file_path)
    print("Base de datos maestra cargada exitosamente.")
except FileNotFoundError:
    print(f"ERROR: No se encontró la base de datos maestra en: {file_path}")
    exit()


# --- 3. AGREGACIÓN POR SECCIÓN ---
print("Agregando datos electorales por sección...")

# Definimos las operaciones de agregación
agregaciones = {
    'lista_nominal': 'mean',
    'voto_total': 'sum',
    'morena_coalitions': 'sum',
    'non_morena_coalitions': 'sum',
    # Usamos una función lambda para obtener el ganador más frecuente (la moda)
    'ganador_casilla_partido': lambda x: x.mode().iloc[0] if not x.mode().empty else 'N/A'
}

# Agrupamos por sección y aplicamos las agregaciones
df_agregado = master_df.groupby('seccion').agg(agregaciones).reset_index()

# Renombramos columnas para mayor claridad
df_agregado = df_agregado.rename(columns={
    'lista_nominal': 'lista_nominal_promedio',
    'voto_total': 'votos_totales_acumulados',
    'morena_coalitions': 'votos_morena_acumulados',
    'non_morena_coalitions': 'votos_oposicion_acumulados',
    'ganador_casilla_partido': 'partido_dominante'
})

# --- 4. CÁLCULO DE NUEVAS MÉTRICAS ---
print("Calculando métricas adicionales (porcentajes, participación)...")

# Calculamos el porcentaje de voto histórico para cada bloque
# Se añade una pequeña cantidad (1) al denominador para evitar divisiones por cero
denominador = df_agregado['votos_morena_acumulados'] + df_agregado['votos_oposicion_acumulados'] + 1
df_agregado['pct_voto_morena'] = (df_agregado['votos_morena_acumulados'] / denominador) * 100

# Calculamos una tasa de participación promedio de forma más precisa
participacion_df = master_df.groupby('seccion')[['voto_total', 'lista_nominal']].sum().reset_index()
participacion_df['tasa_participacion_promedio'] = (participacion_df['voto_total'] / (participacion_df['lista_nominal'] + 1)) * 100
df_agregado = df_agregado.merge(participacion_df[['seccion', 'tasa_participacion_promedio']], on='seccion', how='left')


# --- 5. GUARDADO ---
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_agregado.to_csv(output_path, index=False)

print(f"\n¡Base de datos agregada por sección creada exitosamente!")
print(f"Guardada en: {output_path}")
print("\nResumen de la base de datos agregada:")
df_agregado.info()
print("\nPrimeras filas de la base de datos agregada:")
display(df_agregado.head())